# Failed Jobs via Held Logs

In [96]:
import re
import numpy as np
import datetime

new_ram_reqs = 8
jobs_log_file = './jobs.log'
dag_file = './aldd_dag.dag'
rnd_job_name_fmt='RND_{}_{}_{}'
dist_job_name_fmt='DIST_{}_{}_{}'
exploration_strategy = ['weighted', 'random', 'dissimilar']
rnd_flag = '--random_param_sampling'
dist_flag = '--no-random_param_sampling'

with open(jobs_log_file) as f:
    jobs_log = f.readlines()
jobs_log = [x.strip() for x in jobs_log] 
jobs_log = jobs_log[1:]

with open(dag_file) as f:
    dag_log = f.readlines()
dag_log = [x.strip() for x in dag_log] 

job_names_1 = []
for i, x in enumerate(jobs_log):
    job_split = jobs_log[i].split()
    process_num = job_split[13]
    batch_size_index = job_split[14]
    exploration_strategy = job_split[11]
    sampling_type = job_split[16]
    
    if rnd_flag == sampling_type:
        job_names_1.append(rnd_job_name_fmt.format(process_num, batch_size_index, exploration_strategy))
    else:
        job_names_1.append(dist_job_name_fmt.format(process_num, batch_size_index, exploration_strategy))

new_dag_log = []
for i in range(len(job_names_1)):
    new_dag_job_entry = [x for x in dag_log if job_names_1[i] in x]
    vars_idx, vars_entry = [(xi, x) for xi, x in enumerate(new_dag_job_entry) if 'VARS {}'.format(job_names_1[i]) in x][0]
    vars_entry_split = vars_entry.split()
    vars_entry_split[-2] = 'ramreqs=\"{}\"'.format(new_ram_reqs)
    vars_entry = ' '.join(vars_entry_split)
    new_dag_job_entry[vars_idx] = vars_entry
    new_dag_log.extend(new_dag_job_entry)
    
new_dag_log.extend(['RETRY ALL_NODES 5', 'CONFIG my.dag.config'])

with open('aldd_dag_1.dag', 'w') as f:
    for item in new_dag_log:
        f.write("%s\n" % item)

---
# Failed Jobs via ./params_results success_flag.txt

In [102]:
import re
import numpy as np
import datetime
import glob
import os

rnd_job_name_fmt='RND_{}_{}_{}'
dist_job_name_fmt='DIST_{}_{}_{}'
exploration_strategy = ['weighted', 'random', 'dissimilar']
batch_size_index_dict = {96: 0, 384: 1, 1536: 2}

job_names_2 = []
params_dir = './params_results/*/*/*/*/'
for d in glob.glob(params_dir):
    sampling_type, exploration_strategy, process_num, batch_size_index = d.split('\\')[1:-1]
    batch_size_index = batch_size_index_dict[int(batch_size_index[11:])]
    process_num = int(process_num[23:])
    
    with open(d+'success_flag.txt') as f:
        success_flag = int(f.readlines()[0][0])
    
    if not success_flag:
        if sampling_type == 'random':
            job_names_2.append(rnd_job_name_fmt.format(process_num, batch_size_index, exploration_strategy))
        else:
            job_names_2.append(dist_job_name_fmt.format(process_num, batch_size_index, exploration_strategy))
            
dag_file = './aldd_dag.dag'
with open(dag_file) as f:
    dag_log = f.readlines()
dag_log = [x.strip() for x in dag_log] 

new_dag_log = []
for i in range(len(job_names_1)):
    new_dag_job_entry = [x for x in dag_log if job_names_1[i] in x]
    vars_idx, vars_entry = [(xi, x) for xi, x in enumerate(new_dag_job_entry) if 'VARS {}'.format(job_names_1[i]) in x][0]
    vars_entry_split = vars_entry.split()
    vars_entry_split[-2] = 'ramreqs=\"{}\"'.format(new_ram_reqs)
    vars_entry = ' '.join(vars_entry_split)
    new_dag_job_entry[vars_idx] = vars_entry
    new_dag_log.extend(new_dag_job_entry)
    
new_dag_log.extend(['RETRY ALL_NODES 5', 'CONFIG my.dag.config'])

with open('aldd_dag_1.dag', 'w') as f:
    for item in new_dag_log:
        f.write("%s\n" % item)

In [1]:
import numpy as np

a = np.array([98088, 43063, 47077,  3336, 37254, 55901, 22899, 30816, 25349, 73503, 36375, 13116,
 24855, 15314, 30516, 500,  5826, 55496, 32248, 90093, 28205, 55373, 68909, 46852,
 81062, 84905, 64208, 81894, 43486, 70805, 89341, 44552, 63467,  2317,  4545, 29547,
 24713, 47398,  8739, 36044, 17506, 91962, 15909, 42747, 23978, 17977, 87044,  9420,
 39870, 51334, 71180, 32812, 88466, 36281 , 9384,  8733, 64197, 23493, 84220, 28561,
 61239, 40146, 74714,  4327, 85404, 91523, 88544, 61333, 72157, 67816,  3322, 55167,
 81592,  4989, 65881, 86224, 37589, 35410, 82455, 35475,  8656, 13181, 22910, 44652,
 75992, 35960, 38015, 63649, 41833, 16405, 65276, 87714,  2910, 29051, 56044, 53101])

In [2]:
a[a >= 94761]

array([98088])

In [3]:
94761 + 96

94857

In [4]:
a.shape

(96,)

In [12]:
a = []
a.append((1,))
a[-1] = a[-1] + (3,)

In [13]:
a

[(1, 3)]

In [26]:
import pandas as pd
df = pd.concat([pd.read_csv('../datasets/lc_clusters_cv_96/unlabeled_{}.csv'.format(i)) for i in range(1, 989)])

In [34]:
np.where(df['Cluster_0.4'] == 108254)

(array([50412], dtype=int64),)

In [36]:
98088 - 50412

47676